In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

van Der Waals correction will be unavailable. Please install ased3


In [2]:
dir = '/home/jujuman/Research/MD_TEST/2luf_big/'

# Molecule file
molfile = dir + '2luf_solv.pdb'
#molfile = dir + 'optmol.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'optmol.xyz'

T = 300.0 # Temperature
dt = 0.5
C = 0.1 # Optimization convergence
steps = 1000

ntdir = '/home/jujuman/Research/ANI-AL_retrain/arc3/'
cns = ntdir + 'rHCNO-4.6R_16-3.1A_a4-8_3.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 1

In [3]:
# Load molecule
mol = read(molfile)
L = 100.00
mol.set_cell(([[L, 0, 0],
               [0, L, 0],
               [0, 0, L]]))

mol.set_pbc((True, True, True))

In [4]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

# Set ANI calculator
mol.set_calculator(ANIENS(aens,sdmx=20000000.0))

In [ ]:
def erg_rsix (Rmag):
    return np.power(Rmag,-4)

def frc_rsix (Rmag,R):
    F = 4.0*np.power(Rmag,-6)*R                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    return F

#mol.calc.set_pairwise(erg_rsix, frc_rsix)

In [ ]:
# Optimize molecule
start_time = time.time()
dyn = QuasiNewton(mol)
dyn.run(fmax=C)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)

hdt.writexyzfile(optfile, pos, spc)

      Step     Time          Energy         fmax
FIRE:    0 11:49:17 -48307981.485590      244.8915


In [ ]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = Langevin(mol, dt * units.fs, T * units.kB, 0.04)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions()
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
dyn.attach(storeenergy, interval=100)

In [ ]:
start_time = time.time()
dyn.run(steps)
print('[ANI Total time:   ', time.time() - start_time, 'seconds]')
print('[ANI per step time:', 1000.0*(time.time() - start_time)/steps, 'ms]')